## Phantom Data: Type State Pattern, Zero Size Types
---

In [2]:
#[derive(Debug)]
struct Locked;
#[derive(Debug)]
struct Unlocked;

In [3]:
#[derive(Debug)]
struct Database<LockState = Locked> {
    key: String,
    db: std::collections::HashMap<String, String>,
    lock_state: std::marker::PhantomData<LockState>,
}

In [4]:
impl Database<Locked> {
    fn unlock(self, key: impl Into<String>) -> Database<Unlocked> {
        Database {
            key: key.into(),
            db: self.db,
            lock_state: std::marker::PhantomData::<Unlocked>,
        }
    }
}

In [5]:
impl Database<Unlocked> {
    fn lock(self) -> Database<Locked> {
        Database {
            key: self.key,
            db: self.db,
            lock_state: std::marker::PhantomData::<Locked>,
        }
    }

    fn get(&self, akey: String) -> Option<String> {
        self.db.get(&akey).cloned()
    }
}

In [6]:
impl<LockState> Database<LockState> {
    fn version(&self) -> u64 {
        0u64
    }
}

In [7]:
impl Database {
    fn new(key: impl Into<String>) -> Self {
        Database {
            key: key.into(),
            db: Default::default(),
            lock_state: Default::default(),
        }
    }
}

In [8]:
let db = Database::new("test");
db

Database { key: "test", db: {}, lock_state: PhantomData<ctx::Locked> }

In [9]:
let db = db.unlock("test");
db

Database { key: "test", db: {}, lock_state: PhantomData<ctx::Unlocked> }

In [10]:
db.lock()

Database { key: "test", db: {}, lock_state: PhantomData<ctx::Locked> }